# Live-Data
- Web-Scraping
- Jede Stunde werden die Angaben der __LIVE.csv__ Tabelle aktualisiert
#### Was noch?
- Manche Daten sind allerdings festgelegt.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
import time
import pytz
import pandas as pd
import numpy as np

ldf = pd.DataFrame({"TimeStamp": [],
                    "Uhrzeit": [], "Jahreszeit": [], 
                    "Event": [],"Ferien": [], "Feiertag": [], 
                    "Temp": [], "Rain": [], "Wind": []})


In [2]:
csv_name="LIVE.csv"

In [3]:
def go_Live():
    # WEBSCRAPING
    url_wetterdaten = 'http://wetter.upb.de/handy.html/'
    response_wd = requests.get(url_wetterdaten)
    soup_wd = BeautifulSoup(response_wd.content, "html5lib")

    res = requests.get(url_wetterdaten)
    html_page = res.content
    soup_wd = BeautifulSoup(html_page, 'html.parser')
    text = soup_wd.find_all(text=True)

    #Getter von live-Wetterdaten
    def get_temp():
        for i,t in enumerate(text):
            if 'Temp:' in t:
                x = i+1
        z = ''
        for i in text[x]:
            if i.isnumeric() == True or i == '.':
                z = z + i
        return z
    def get_rain():
        for i,t in enumerate(text):
            if 'Regen' in t:
                x = i+1
        z = ''
        for i in text[x]:
            if i.isnumeric() == True or i == '.':
                z = z + i
        return z

    def get_wind():
        for i,t in enumerate(text):
            if 'Windstärke' in t:
                x = i+2
            z = ''
        for i in text[x]:
            if i.isnumeric() == True or i == '.':
                z = z + i
        return z

    def get_zeit() -> dt.datetime:
        tz= pytz.timezone("Europe/Berlin")
        uhrzeit = tz.localize(dt.datetime.now()).replace(microsecond=0, second=0, minute=0)
        return uhrzeit
    
    def get_wochentag():
        tag = get_zeit().weekday()
        
        if tag == 0:
            _tag = 'Montag'
        elif tag == 1:
            _tag = 'Dienstag'
        elif tag == 2:
            _tag = 'Mittwoch'
        elif tag == 3:
            _tag = 'Donnerstag'
        elif tag == 4:
            _tag = 'Freitag'
        elif tag == 5:
            _tag = 'Samstag'
        elif tag == 6:
            _tag = 'Sonntag'
        else : 
            pass 

        return _tag

    def get_jahreszeit():
        tag = get_zeit()
        if 3 <= tag.month <= 5:
            return 'Frühling'
        elif 6 <= tag.month <= 8:
            return 'Sommer'
        elif 9 <= tag.month <= 11:
            return 'Herbst'
        elif 12 == tag.month or 1 <= tag.month <= 2:
            return 'Winter'


    event = 'nichts'
    feiertag = False
    ferien = False
    
    return {
        'Uhrzeit': get_zeit().strftime("%H:%M:%S"),
        'Wochentag': get_wochentag(),
        'Jahreszeit': get_jahreszeit(),
        'Events': event,
        'Ferien': ferien,
        'Temp': get_temp(),
        'Rain': get_rain(),
        'Windlage': get_wind(),
        'Feiertag': feiertag
    }


In [4]:
def fetch_data():
    data = go_Live()
    frame = pd.DataFrame(data=data, index=[0])
    frame.to_csv(csv_name, index=False)

In [5]:
fetch_data()

while True:
    time.sleep(0.5)
    now = dt.datetime.now()
    if now.minute == 1:
        break
while True:
    fetch_data()
    time.sleep(3600)

KeyboardInterrupt: 